In [1]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup

In [2]:
# convert model names to brands

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
#file = open('models.csv', 'r')
file = open('models.csv', 'r', encoding='mac_roman', newline='')

csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    brands.append(i)
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [3]:
brands

[['acura', 'integra'],
 ['acura', 'Legend†'],
 ['acura', 'vigor'],
 ['acura', 'rlx'],
 ['acura', 'ILX'],
 ['acura', 'MDX'],
 ['acura', 'RDX'],
 ['acura', 'TSX'],
 ['acura', 'RSX'],
 ['audi', 'quattro'],
 ['audi', 'a4'],
 ['audi', 'a6'],
 ['audi', 'a5'],
 ['audi', 'a7'],
 ['audi', 'q3'],
 ['audi', 'q5'],
 ['audi', 'q7'],
 ['audi', 'a8'],
 ['audi', 'a3'],
 ['bmw', '3-series'],
 ['bmw', '5-series'],
 ['bmw', '6-series'],
 ['bmw', '4-series'],
 ['bmw', 'xdrive'],
 ['bmw', 'alpina'],
 ['bmw', '330ci'],
 ['bmw', '328i'],
 ['bmw', '330c'],
 ['bmw', '335i'],
 ['bmw', '335d'],
 ['bmw', '528i'],
 ['bmw', '535i'],
 ['bmw', '128i'],
 ['bmw', '640i'],
 ['bmw', '3er'],
 ['bmw', '5er'],
 ['bmw', 'M5'],
 ['bmw', 'M3'],
 ['bmw', 'M6'],
 ['bmw', 'bmws'],
 ['bmw', 'bmwesque'],
 ['buick', 'park avenue'],
 ['buick', 'la crosse'],
 ['buick', 'lacrosse'],
 ['buick', 'century'],
 ['buick', 'enclave'],
 ['buick', 'encore'],
 ['buick', 'regal'],
 ['buick', 'buicks'],
 ['cadillac', 'escalade'],
 ['cadillac', 'sr

## Scraping all pages

In [4]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [5]:
# number of forum pages
num_pages = 2572

In [ ]:
# store the comments in a string
all_comments = ""

# store output: date, userid, message
output_csv = []

# go through each page
for page_number in range(1, 200):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
    
        # get the html of the page in string form
        page_html = page.text
    
        # create the BeautifulSoup object that takes in the html in str form and a html/xml
        #parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
    
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
        
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        #usermessages = messagelist.find_all(class_ =  "Message userContent")
        usermessages = messagelist.find_all(class_ = "Comment")
        
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # let's grab the date and username of the post which is under class = "Item-Header CommentHeader"
            # in a <time> tag and class = "Username"
            commentheader = usermessages[i].find(class_ = 'Item-Header CommentHeader')
            
            # grab the date. Format: June 18, 2014  3:53PM is stored in title = ' ' of <time  >
            date_of_post = commentheader.find("time", title = True)
            
            # variable to store date
            date_of_post = date_of_post['title']
            
            # grab the username
            username = commentheader.find(class_ = "Username")
            # variable to store username
            username = username.string
            
            # now grab text under Message userContent
            messageheader = usermessages[i].find(class_ = 'Message userContent')
            
            # a user comment might have multiple <p> tags
            # [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            [s.extract() for s in messageheader(class_ = 'UserQuote')]
            user_comments = messageheader.find_all('p')
            
            # variable to store user message
            message = ""
            for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
                [s.extract() for s in user_comments[j]('a')]
                [s.extract() for s in user_comments[j]('img')]
                [s.extract() for s in user_comments[j]('br')]

                
                # Let's do some string manipulation 
                # remove the newline characters, '<p>', '</p','said:'  
                # add the users comments to the mega string

                message += user_comments[j].prettify()

            # let's do some string manipulation
            # remove the newline characters, '<p>', '</p','said:'
            message = message.replace('\n', '')
            message = message.replace(r"<p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"said:", "")
            message = message.replace(r":", "")
            message = message.lower()
            
            # deal with brand name
            for k in brands:
                message = message.replace(k[1], k[0])

            # add to all_comments
            all_comments += message
            
            # create a tuple with Date, Userid, Message and store into output_csv
            row = date_of_post, username, message
            output_csv.append(row)
            
            # print this if you want to see some output
            # print('username: ' + str(username))
            # print('date: ' + str(date_of_post))
            # print('message: ' + message)
            
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete
https://forums.edmunds.com/disc

In [12]:
output_csv[0:2]

[('June 18, 2014  3:53PM',
  'graphicguy',
  " i'm 2nd post....yee haw!!!!!!!! do i get a badge?"),
 ('June 18, 2014  4:01PM',
  'bwia',
  ' great, but will it be possible to refer to old thread? and one more thing how do i find and add a discussion to "my bookmarks?"')]

### Throw output_csv list into a csv file

In [13]:
# throw this into a text file just in case
with open('comments.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['date','username', 'comment'])
    for row in output_csv:
        csv_out.writerow(row)

### Get list of comments

In [53]:
comments = []

In [54]:
for c in output_csv:
    comments.append(c[2])

In [70]:
comments[0]

" i'm 2nd post....yee haw!!!!!!!! do i get a badge?"

### Get total number of comments

In [71]:
tot_number_of_comments = len(comments)

### Get list of unique brands

In [85]:
unique_brands = []
for b in brands:
    if b[0] not in unique_brands:
        unique_brands.append(b[0])
unique_brands.remove('car')
unique_brands.remove('hyndai kia')
unique_brands.remove('hyundai,')
unique_brands.remove('hyundai.')
unique_brands.remove('kia,')
unique_brands.remove('kia.')
unique_brands.remove('mercedes benz')
unique_brands.remove('mercedes-benz')
unique_brands.remove('nissan.')
unique_brands.remove('problem')
unique_brands.remove('seat')
unique_brands.remove('toyata')
unique_brands.remove('volkwagen')

In [87]:
unique_brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'saturn',
 'sedan',
 'subaru',
 'suzuki',
 'toyota',
 'volkswagen',
 'volvo']

### Get list of tuple of brand combinations

In [89]:
brand_combos = []
for i in range (len(unique_brands)):
    for j in range (len(unique_brands)):
        if j > i:
            brand_combo = (unique_brands[i], unique_brands[j])
            brand_combos.append(brand_combo)

In [90]:
brand_combos

[('acura', 'audi'),
 ('acura', 'bmw'),
 ('acura', 'buick'),
 ('acura', 'cadillac'),
 ('acura', 'chevrolet'),
 ('acura', 'chrysler'),
 ('acura', 'dodge'),
 ('acura', 'ford'),
 ('acura', 'honda'),
 ('acura', 'hyundai'),
 ('acura', 'infiniti'),
 ('acura', 'kia'),
 ('acura', 'lincoln'),
 ('acura', 'mazda'),
 ('acura', 'mercedes'),
 ('acura', 'mercury'),
 ('acura', 'mitsubishi'),
 ('acura', 'nissan'),
 ('acura', 'pontiac'),
 ('acura', 'saturn'),
 ('acura', 'sedan'),
 ('acura', 'subaru'),
 ('acura', 'suzuki'),
 ('acura', 'toyota'),
 ('acura', 'volkswagen'),
 ('acura', 'volvo'),
 ('audi', 'bmw'),
 ('audi', 'buick'),
 ('audi', 'cadillac'),
 ('audi', 'chevrolet'),
 ('audi', 'chrysler'),
 ('audi', 'dodge'),
 ('audi', 'ford'),
 ('audi', 'honda'),
 ('audi', 'hyundai'),
 ('audi', 'infiniti'),
 ('audi', 'kia'),
 ('audi', 'lincoln'),
 ('audi', 'mazda'),
 ('audi', 'mercedes'),
 ('audi', 'mercury'),
 ('audi', 'mitsubishi'),
 ('audi', 'nissan'),
 ('audi', 'pontiac'),
 ('audi', 'saturn'),
 ('audi', 'seda

### Get dictionary of:
- {brand_combo: number of comments mentioning both brands}

In [91]:
brand_combo_dict = {}
for bc in brand_combos:
    for c in comments:
        brand1 = bc[0]
        brand2 = bc[1]
        if brand1 in c and brand2 in c:
            if bc in brand_combo_dict:
                brand_combo_dict[bc] += 1
            else:
                brand_combo_dict[bc] = 1

In [92]:
brand_combo_dict

{('acura', 'bmw'): 1,
 ('bmw', 'ford'): 1,
 ('bmw', 'mercedes'): 1,
 ('bmw', 'volvo'): 1,
 ('ford', 'mercedes'): 1}

- {unique_brand: number of comments mentioning brand}<br>

In [93]:
unique_brands_dict = {}
for b in unique_brands:
    for c in comments:
        if b in c:
            if b in unique_brands_dict:
                unique_brands_dict[b] += 1
            else:
                unique_brands_dict[b] = 1

In [95]:
unique_brands_dict

{'acura': 1,
 'bmw': 4,
 'buick': 1,
 'chevrolet': 1,
 'dodge': 1,
 'ford': 1,
 'honda': 1,
 'mercedes': 1,
 'nissan': 2,
 'volvo': 2}

### Calculate lift for each brand combo

In [96]:
# tot_number_of_comments
# unique_brands_dict
# brand_combo_dict

brand_combo_lifts_dict = {}
for bc in brand_combo_dict:
    
    combo_count = brand_combo_dict[bc]
    brand1_count = unique_brands_dict[bc[0]]
    brand2_count = unique_brands_dict[bc[1]]
    
    lift = (tot_number_of_comments * combo_count) / (brand1_count * brand2_count)
    
    brand_combo_lifts_dict[bc] = lift

In [97]:
brand_combo_lifts_dict

{('acura', 'bmw'): 15.0,
 ('bmw', 'ford'): 15.0,
 ('bmw', 'mercedes'): 15.0,
 ('bmw', 'volvo'): 7.5,
 ('ford', 'mercedes'): 60.0}